In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import csv
import json

In [ ]:
with open('data/dk_asylum_data.json', encoding='utf-8') as inputfile:
    df = pd.read_json(inputfile)

## Extract decision

In [ ]:
# affirmed means the Immigration service decision was confirmed and the asylum application was rejected, overturned means asylum was granted.
#this should however be checked with e.g. Trine.
df_aff = df_raw['text'].str.contains('stadfæst')
df_rej = df_raw['text'].str.contains('Nævnet omgjorde')
df_med = df_raw['text'].str.contains('Nævnet meddelte')
df_hjem = df_raw['text'].str.contains('Nævnet hjemvis')
df_and = df_raw['text'].str.contains('Nævnet ændrede')

df_leftover = df_raw[~(df_aff | df_rej | df_med | df_hjem | df_and)]
print(len(df_leftover))

df['decision'] = np.where(df_aff, "affirmed", np.where((df_rej | df_and | df_med), "overturned", np.nan))

## Extract year of entry

In [ ]:
df['entry_year'] = df['text'].str.extract(r'.*(Indrejst(\D*)(\d+))')[2]

## Extract decision year

In [ ]:
u = df['text'].str.partition()
df_extract = pd.DataFrame({'Last': u[2].str.split().str[-1]})
df['decision_year'] = df_extract['Last'].str.extract(r'.*([1-3][0-9]{3})')

## Extract gender

In [ ]:
#there is a significant amount of cases where families apply. These can be identified by searching if
#"A/ansøgerne" appears in the text
def get_gender(text):
    fem_count = 0
    mal_count = 0
    gender_dict = {
        "female": ['kvinde', 'kvindelig', 'pige'],
        "male": ['mand', 'mandlig', 'dreng'],
    }
    for word in gender_dict['female']:
        if word in text:
            fem_count += 1
    for word in gender_dict['male']:
        if word in text:
            mal_count += 1
    
    #print(fem_count, mal_count)
    if fem_count > mal_count:
        return "female"
    elif fem_count < mal_count:
        return "male"
    else:
        return "nan"
        
    

df["gender"] = df["text"].apply(get_gender)